In [52]:
import os
import numpy as np
import pandas as pd


data_path = '/media/donghyun/Samsung_T5/ship_data'

raw_data_path = os.path.join(data_path, '3.csv_1m_cut')
labels_path = os.path.join(data_path, '5km_ship_label.csv')

    
# labels load

labels_df = pd.read_csv(os.path.join(labels_path))
temp = labels_df.to_numpy()
labels = np.delete(temp, 1, axis = 1)    # B열 제거

raw_data_file_list = []
for file_name in os.listdir(raw_data_path):
    raw_data_file_list.append(file_name)


print(labels.shape)
print(len(raw_data_file_list))

(15849, 1)
14633


In [53]:
# Divide from data to true and false data set

true_data_indices = []
false_data_indices = []

for i in range(len(labels)):
    
    if labels[i] == 1 :
        true_data_indices.append(i)
    else :
        false_data_indices.append(i)

        
print(len(true_data_indices))
print(len(false_data_indices))


1706
14143


In [55]:
# Divide train (0.8) and validation (0.2) index
# True data (train 0.8, val 0.2)
# False data (train 0.8 , val 0.2)


from sklearn.model_selection import train_test_split

true_train, true_val = train_test_split(true_data_indices, test_size = 0.2)
false_train, false_val = train_test_split(false_data_indices, test_size = 0.2)

print(len(true_train))
print(len(true_val))
print(len(false_train))
print(len(false_val))


1364
342
11314
2829


In [56]:
# make chunk

def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

chunked_false_train = list_chunk(false_train, len(true_train))
chunked_false_val = list_chunk(false_val, len(true_val))

chunked_false_train = chunked_false_train[:-1]
chunked_false_val = chunked_false_val[:-1]

print(np.array(chunked_false_train).shape)
print(np.array(chunked_false_val).shape)


(8, 1364)
(8, 342)


In [66]:
# tensorflow model structure

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Input, Activation, BatchNormalization, concatenate, Dropout
from tensorflow.keras.layers import average, Add, Reshape, multiply

from tensorflow.keras.optimizers import Adam


def block(x):
    
    x = Conv1D(32, 5, strides = 1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(16, 5, strides= 1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv1D(8, 5, strides= 1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x


def Model(input_shape):
    
    input_ = Input(shape = input_shape)
    
    X = block(input_)
    X = block(X)
    
    
    X = Dense(units=100, activation='relu')(X)
    output_ = Dense(units = 2, activation='sigmoid')(X)
    
    model = Model(inputs = input_, outputs = output_)
    
    return model


In [67]:
# kernel init

tf.keras.backend.clear_session()

In [68]:
# model papams & compile

input_shape = (12001, 119)

opt = Adam(learning_rate = 0.001)

clf_model = Model(input_shape)
clf_model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['acc'])

clf_model.summary()

TypeError: Model() got an unexpected keyword argument 'inputs'

In [65]:
print(np.array(chunked_false_train).shape)
print(np.array(chunked_false_val).shape)

print(np.array(true_train).shape)
print(np.array(true_val).shape)


(8, 1364)
(8, 342)
(1364,)
(342,)


In [74]:
np.array(chunked_false_train)


array([[11811, 14029,  6507, ...,  2172,   507, 11079],
       [13071,  9380,   293, ...,  6718, 11417,  1665],
       [ 2500,  7109, 12225, ..., 14553, 10227, 11155],
       ...,
       [ 4023, 12925, 14554, ...,   621,  3866, 15505],
       [ 5102,   804,  4999, ...,   273, 12496,  8975],
       [ 3502, 10624, 13391, ...,  6346,  6957,  8033]])

In [75]:

# load true data

true_train_data = []
true_train_labels = [1 for i in range(len(true_train))]
for index in true_train:
    print(index)

    df = pd.read_csv(os.path.join(raw_data_path + '/' + raw_data_file_list[index]))
    data = df.to_numpy()

    true_train_data.append(data)


true_val_data = []
true_val_labels = [1 for i in range(len(true_val))]
for index in true_val:
    print(index)

    df = pd.read_csv(os.path.join(raw_data_path + '/' + raw_data_file_list[index]))
    data = df.to_numpy()

    true_val_data.append(data)


print(np.array(true_train_data).shape)
print(np.array(true_val_data).shape)
    
### train session

EPOCH = 50

# train log
train_acc_list = []
train_loss_list = []

# validation log
val_acc_list = []
val_loss_list = []


for epoch in range(EPOCH):
    break
    
    print('epoch : {}'.format(epoch+1))
    
    
    # load false data
    
    for nChunk in range(len(chunked_false_train)):   # 8
        
        chunked_false_train_data = []
        chunked_false_train_labels = [0 for i in range(len(chunked_false_train[nChunk]))]
        for index in chunked_false_train[nChunk]:
        
            df = pd.read_csv(os.path.join(raw_data_path + '/' + raw_data_file_list[index]))
            data = df.to_numpy()
        
            chunked_false_train_data.append(data)  
    
    
        chunked_false_val_data = []
        chunked_false_val_labels = [0 for i in range(len(chunked_false_val[nChunk]))]
        for index in chunked_false_val[nChunk]:
        
            df = pd.read_csv(os.path.join(raw_data_path + '/' + raw_data_file_list[index]))
            data = df.to_numpy()
        
            chunked_false_val_data.append(data)
    
        print(np.array(chunked_false_train_data).shape)
        print(np.array(chunked_false_val_data).shape)
        print(np.array(true_val_data).shape)
        print(np.array(true_train_data).shape)

        
        # get batch
        train_data = true_train_data + chunked_false_train_data
        val_data = true_val_data + chunked_false_val_data
    
        train_labels = true_train_labels + chunked_false_train_labels
        val_labels = true_val_labels + chunked_false_val_labels
    
    
        batched_train_set = tf.data.Dataset.from_tensor_slices(train_data, train_labels).shuffle(20000).batch(64)
        batched_val_set = tf.data.Dataset.from_tensor_slices(val_data, val_labels ).shuffle(10000).batch(16)
    
    
        break

        ## training
        
        train_acc = 0.0
        train_loss = 0.0

        i = 0
        for batch, data in enumerate(batched_train_set):

            i = batch+1

            train_data, label = data

            batch_loss = clf_model.train_on_batch(train_data, label)

            train_acc += batch_loss[1]
            train_loss += batch_loss[0]

        train_acc_list.append(train_acc/i)
        train_loss_list.append(train_loss/i)


        ## validation

        val_acc = 0.0
        val_loss = 0.0

        i = 0
        for batch, data in enumerate(batched_val_set):

            i = batch+1

            val_data, label = data

            batch_loss = clf_model.test_on_batch(val_data, label)

            val_acc += batch_loss[1]
            val_loss += batch_loss[0]


        val_acc_list.append(val_acc/i)
        val_loss_list.append(val_loss/i)


        print("| train acc : {:.4f}, train loss : {:.4f}, val acc : {:.4f}, val loss : {:.4f} |".format(
            (train_acc/i), (train_loss/i), (val_acc/j), (val_loss/j)))

        print()

2254
7667
2125
5570
5451
3578
6190
2255
324
5533
4855
9014
2918
1290
7203
2248
3215
5016
5583
4865
7696
1276
5523
2030
5397
2140
317
4867
4203
6310
1982
971
6309
4011
4537
5008
3194
6292
1597
7322
952
4351
7659
4118
721
4549
1797
1097
4521
3429
3198
7815
1372
4765
686
7201
4102
6003
5454
2244
4256
1877
4870
5512
4853
3556
5515
5439
2940
1965
2285
4233
168
1902
5481
6195
5642
6006
6177
7998
9020
953
4656
4658
4839
5054
2272
681
967
2032
3455
5060
1273
4517
4107
4781
6874
2663
986
6893
7820
3576
6002
1799
4252
2897
7025
717
4121
3305
6005
1103
3156
7825
5646
2048
7753
5644
2279
4523
3312
4516
6311
2266
7669
9008
9097
4257
7200
7336
723
3453
991
4105
5449
1806
3221
4109
4519
3191
3574
7810
7811
694
1897
8335
4224
590
4843
588
7062
1896
1591
3999
7328
4237
7050
1816
2043
5077
7021
2031
2642
6308
6007
7700
4835
6315
1286
7207
4777
4553
185
5992
954
2904
2166
3456
5056
2132
4858
8330
1882
7668
2896
6279
5010
4245
6184
389
687
5647
2281
5076
2898
1374
974
2046
7057
2027
5069
9136
5469
1608
18

IndexError: list index out of range